<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Danilo\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [52]:
# Cargar datos
path = 'C:/Users/Danilo/Documents/KonradLorenz/NLP/Taller9/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [53]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto if palabra not in stopwords]
    #texto = " ".join(texto)
    return texto

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [54]:
dictionary = Dictionary(data['pp'].values)
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['pp'].values]

In [75]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 3, passes=20) 
#De indice a palabra, Numero de temas, cuantas veces repasa el modelo los datos.

###  `[25 pts]` Punto 3: Visualización de LDA

In [76]:
model.print_topics(num_words=5) #Imprime las 5 palabras mas importantes de cada topics 

[(0,
  '0.042*"calidad" + 0.035*"precio" + 0.028*"bien" + 0.027*"producto" + 0.023*"buena"'),
 (1,
  '0.041*"bien" + 0.018*"si" + 0.017*"bonitos" + 0.014*"esperaba" + 0.012*"pendientes"'),
 (2,
  '0.031*"juego" + 0.023*"si" + 0.013*"amazon" + 0.013*"bonito" + 0.012*"pulsera"')]

In [77]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

**RTA/=**
Que siempre en lo posible que se tengan las etiquetas los modelos supervisados serán mejores, pero en la vida real muchas veces no sucede eso, al evaluar el modelo k means que no teniamos las etiquetas, las métricas empeoraron, de igual forma al hacer un modelo de LDA y explorar el número de topics se encuentra un valor cualitativo por medio de la gráfica de 3, pero ya sabemos que el valor exacto de grupos es de 2 (videojuegos y joyeria), entonces pueden haber una mejora en la clusterización de los datos.